# Tarea5: Series de Taylor

Fecha de envío del PR inicial: **viernes 21 de abril**

Fecha de aceptación del PR: **martes 25 de abril, antes de la clase**

---

## Ejercicio 1

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests_taylor.jl") para cada uno de los métodos que implementen.


In [1]:
"""
Definición de polinomios de Taylor, donde esta compuesto por un numero que denota el orden (es de tipo entero)
dado por un valor n y un vector de un orden n+1
"""
type Taylor{T<:Number}
    orden::Int #el orden de la serie de taylor.
    coff::Array{T,1}   #coeficientes en la serie de taylor.
    function Taylor(orden::Int,coff::Array{T,1})#ESTE AGREGADO ES NECESARIO PARA GARANTIZAR QUE TODOS LOS ELEMENTOS INGRESADOS EN EL VECTOR SEAN DEL MISMO ORDEN QUE LA N PROPUESTA
        if length(coff)==orden+1
            new(orden,coff) 
        else
        error("función no válida ya que el orden no coincide con el número de coeficientes")
        end
    end
end
#se especifican los posibles casos para los que se puede usar este taylor.
Taylor{T<:Number}(orden::Int,coff::Array{T,1})=Taylor{T}(orden,coff)    #el caso general
Taylor{T<:Number}(coff::Array{T,1})=Taylor{T}(length(coff-1),coff)
Taylor{T<:Number}(orden::Int,coff::T)=Taylor{T}(orden,[coff])    #este caso particular es para escalares 

Taylor{T<:Number}

In [2]:
a=[1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [3]:
Taylor(1,a)

LoadError: función no válida ya que el orden no coincide con el número de coeficientes

In [4]:
Taylor(2,a)

Taylor{Int64}(2,[1,2,3])

Esto sólo es un ejemplo para ver que el nuevo type *Taylor* funciona correctamente

In [5]:
Taylor(0,1)

Taylor{Int64}(0,[1])

Este Taylor nos da el caso de una función del tipo constante, ya que éstas son de orden cero

In [6]:
function igualavector!(a,b)    #esta funcion compara los valores de los órdenes de los polinomios
    if length(a.coff) < length(b.coff)  
        for i in length(a.coff):length(b.coff)-1    
            push!(a.coff,0)
        end
        a.orden=b.orden
        return a,b      #sea cualquier caso, de no ser del mismo orden, el de orden menor aumenta su grado para ser igual al de grado mayor
    elseif length(a.coff) >length(b.coff)
        for i in length(b.coff):length(a.coff)-1
            push!(b.coff,0)
        end 
        b.orden=a.orden
        return a,b
    end
end

igualavector! (generic function with 1 method)

In [7]:
j=Taylor(0,1)
k=Taylor(1,[1,2])

Taylor{Int64}(1,[1,2])

In [8]:
igualavector!(j,k)

(Taylor{Int64}(1,[1,0]),Taylor{Int64}(1,[1,2]))

In [9]:
import Base:+,-,*,/,==   ##importando las operaciones principales de taylor:
#suma,resta,multiplicacion, división y comparación

In [10]:
function +(f::Taylor,g::Taylor) 
    igualavector!(f,g)  #igualdor de taylor's
    return Taylor(f.orden,f.coff+g.coff)   #operación artimética directa, la resta es análoga
end


+ (generic function with 164 methods)

In [11]:
j+k

Taylor{Int64}(1,[2,2])

La suma funciona correctamente, se procede a hacer la resta.

In [12]:
function -(f::Taylor,g::Taylor)
    igualavector!(f,g)  #se usa el igualador de taylor's
    return Taylor(f.orden,f.coff-g.coff)  
end

- (generic function with 191 methods)

In [13]:
j-k

Taylor{Int64}(1,[0,-2])

In [14]:
j=Taylor(1,[1,1])
k=Taylor(1,[1,1])

Taylor{Int64}(1,[1,1])

Una vez definido la suma y la resta, falta programar la multiplicación, división y comparación

In [15]:
function prod!(f::Taylor,g::Taylor)
    m=Taylor(f.orden+g.orden,zeros(f.orden+g.orden+1))    #taylor auxiliar para poder hacer el producto
    f=igualavector!(f,m)
    g=igualavector!(g,m)     #con esto se tienen vectores de mismo orden y coeficientes
    return f,g
end

prod! (generic function with 1 method)

In [16]:
function *(f::Taylor,g::Taylor)     #con la funcion prod ya formulada, se puede hacer producto ya que son del mismo orden
    prod!(f,g)                    #se usa el promotor de coeficientes
    z=Taylor(f.orden,zeros(f.orden+1)) 
    for i in 1:length(f.coff)       #se usa doble ciclo for para poder hacer el producto por cada coeficiente y depositarl en el resultado
        for j in 1:i
            z.coff[i]+=f.coff[j]*g.coff[i+1-j]
        end
    end
    return z
end
*(A::Real,B::Taylor)=Taylor(B.orden,A*B.coff)
*(A::Taylor,B::Real)=Taylor(A.orden,B*A.coff)

* (generic function with 152 methods)

In [17]:
j*k

Taylor{Float64}(2,[1.0,2.0,1.0])

In [18]:
w=ans

Taylor{Float64}(2,[1.0,2.0,1.0])

In [19]:
2*w

Taylor{Float64}(2,[2.0,4.0,2.0])

In [20]:
w*2

Taylor{Float64}(2,[2.0,4.0,2.0])

In [21]:
"""
la comparacion de taylors
"""
==(A::Taylor,B::Taylor)=(A.orden==B.orden && A.coff==B.coff)

Base.:==

In [22]:
Taylor(1,[1,1])==Taylor(1,[1,1])

true

Ya definido el producto, solo falta la división

In [23]:
function /(f::Taylor,g::Taylor)
    d=f.orden-g.orden   #el discriminante para ver si la división da como resultado un polinomio
    if d<0
        error("división no válida ya que el resultado no es un polinomio de grado mayor a cero")
    end
    #fin del if
    igualavector!(f,g)   #se igualan los coeficientes para poder operar en la división
    cociente=zeros(f.orden+1)
    h=1    #contador que indica donde se empieza a ser el resultado distinto de cero
    for i in 1:length(g.coff)
        if g.coff[i]!=0
            cociente[1]=(f.coff[i])/(g.coff[i])     #es es para hacer la división y evitar dividir coeficientes cero
            h=i
            break
            elseif (i==length(g.coff)) & (g.coff[i]==0)    #en caso de encontrar un coeficiente del denominador con cero, manda un error
            error("polinomio divisor posee un cero")
        end
    end
    #fin del for
    for i in 1:h-1   #usando el contador del ciclo pasado
        if f.coff[i]!=0
            error("no tiene una expansión de Taylor")  #esto es para asegurar que no sea de la forma a^n/b^m donde m>n
        end
    end
    #lo que falta ahora, es hacer el resto de la división si el polinomio cumple con lo estipulado previamente
    #como en el producto, en el cociente se hace doble ciclo for
    for i in h+1:length(cociente)-h
        suma=0 #suma de los resultados que se lleven a cabo en la division
        for j in 1:i-h
            suma=suma+(cociente[j])*g.coff[i-j+1]
        end
        cociente[1+i-h]=(1/g.coff[h])*(f.coff[i]-suma)
    end
    for i in d+2:length(f.coff)
        if cociente[i]==0
            deleteat!(cociente,i)
    end
    end
    return Taylor(d,cociente)
end
/(A::Taylor,B::Real)=Taylor(A.orden,(A.coff)/B)

/ (generic function with 55 methods)

In [24]:
j=Taylor(1,[1,1])
k=Taylor(3,[1,3,3,1])

Taylor{Int64}(3,[1,3,3,1])

In [25]:
k/j

Taylor{Float64}(2,[1.0,2.0,1.0])

In [26]:
Taylor(3,[1,2,3,4])+Taylor(4,[1,2,3,4,0])

Taylor{Int64}(4,[2,4,6,8,0])

In [27]:
# Muestren que su código funciona con tests adecuados
using Base.Test
include("runtest_taylor.jl")

Test Summary:      | Pass  Total
  pruebas realizadas |    9      9


Base.Test.DefaultTestSet("pruebas realizadas",Any[Test Passed
  Expression: Taylor(0,[1]) + Taylor(1,[1,2]) == Taylor(1,[2,2])
   Evaluated: Taylor{Int64}(1,[2,2]) == Taylor{Int64}(1,[2,2]),Test Passed
  Expression: Taylor(1,[1,1]) + Taylor(1,[1,1]) == Taylor(1,[2,2])
   Evaluated: Taylor{Int64}(1,[2,2]) == Taylor{Int64}(1,[2,2]),Test Passed
  Expression: Taylor(0,[1]) - Taylor(1,[1,2]) == Taylor(1,[0,-2])
   Evaluated: Taylor{Int64}(1,[0,-2]) == Taylor{Int64}(1,[0,-2]),Test Passed
  Expression: Taylor(1,[1,1]) - Taylor(1,[1,1]) == Taylor(1,[0,0])
   Evaluated: Taylor{Int64}(1,[0,0]) == Taylor{Int64}(1,[0,0]),Test Passed
  Expression: 3 * Taylor(2,[10,20,30]) == Taylor(2,[30,60,90])
   Evaluated: Taylor{Int64}(2,[30,60,90]) == Taylor{Int64}(2,[30,60,90]),Test Passed
  Expression: Taylor(1,[1,1]) * Taylor(1,[1,1]) == Taylor(2,[1,2,1])
   Evaluated: Taylor{Float64}(2,[1.0,2.0,1.0]) == Taylor{Int64}(2,[1,2,1]),Test Passed
  Expression: Taylor(3,[2,4,6,8]) / 2 == Taylor(3,[1,2,3,4])
   Eva

---

## Ejercicio 2

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas necesarias en `runtest_taylor.jl`.

In [28]:
import Base:exp,log,^,sin,cos   #importando mas funciones
function exp(f::Taylor) #se implementa la función de la exponencial
    v=zeros(Number,f.orden+1)
    v[1]=exp(f.coff[1])
    for i in 2:length(v)
        for j in 1:i-1
            v[i]+=(i-j)*(f.coff[i-j+1])*(v[j])
        end
        v[i]=(v[i])/(i-1)
    end
    return Taylor(length(v)-1,v)
end

exp (generic function with 12 methods)

In [52]:
exp(Taylor(2,[1,1,1]))

Taylor{Number}(2,Number[2.71828,2.71828,4.07742])

In [42]:
function log(f::Taylor)  #se implementa la función de logaritmo
    L=zeros(length(f.coff))
    m=0
    L[1]=log(f.coff[1])
    L[2]=(f.coff[2])/(f.coff[1])
    for i in 2:length(f.coff)-1
        for j in 1:i-1
            m+=j*L[j+1]*(f.coff[i+1-j])
        end
        L[i+1]=(1/f.coff[1])*(f.coff[i+1]-m/i)
    end
    return Taylor(length(L)-1,L)
end
        

log (generic function with 19 methods)

.Taylor) in module Main at In[35]:2 overwritten at In[42]:2.


In [55]:
log(Taylor(1,[1,-1]))

Taylor{Float64}(1,[0.0,-1.0])

In [46]:
function ^(f::Taylor,a::Int)   #funcion que implementa la exponencial
    P=zeros(length(f.coff))
    m=0
    if a==0
        Taylor(length(f.coff),zeros(length(f.coff)))
    else
        P[1]=(f.coff[1])^(a)
        P[2]=a*((f.coff[2])/(f.coff[1]))*P[1]
        for i in 1:length(f.coff)-1
            for j in 1:i-1
                m+=((a*i)-(a+1)*(j))*(f.coff[i+1-j])*(P[j+1])
            end
            P[i+1]=(a)*(f.coff[i+1]/f.coff[1])*P[1]+(1/(i*f.coff[1]))*m
        end
        return Taylor(length(P)-1,P)
    end
end     

^ (generic function with 49 methods)

In [53]:
Taylor(1,[2,0])^2

Taylor{Float64}(1,[4.0,0.0])

In [ ]:
function sin(f::Taylor)   #creando la función de seno
    
end